In [ ]:
#allows imports from other folders in project
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# NBEATS MODEL

## Load Data and Create Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from Models.NBEATS import NBeats
from dataPreprocessing import load_data, prepare_dataloader
from PyTorchModular import train_model, loss_curve
from Evaluation.evaluation_helpers import evaluate_model

# Load Data and Create Model
train_file = "../Data/Train/train1990s.csv"
config = {
    "use_fft": False,  # No FFT needed
    "use_exog": False  # No exogenous variables
}
X_train, y_train, X_valid, y_valid, X_test, y_test, dates, y_scaler = load_data(train_file, config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NBeats(input_size=X_train.shape[1]).to(device)


## Train

In [ ]:
# Prepare DataLoaders
dataloader_train = prepare_dataloader(X_train, y_train)
dataloader_valid = prepare_dataloader(X_valid, y_valid)
dataloader_test = prepare_dataloader(X_test, y_test)

# Training Setup
loss_fn = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.005, weight_decay=5e-7)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

savepath = '../Models/Weights/'

# Train Model
trainingMetadata = train_model(
    model, 
    maxEpochs=120, 
    modelSavePath=savepath, 
    modelName='nbeats', 
    dataLoaderTrain=dataloader_train, 
    dataLoaderValid=dataloader_valid, 
    lossFn=loss_fn, 
    optimizer=optimizer, 
    device=device, 
    scheduler=scheduler
)

## Visualise Output

In [ ]:
# Convert to PyTorch Tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_exog_train_tensor = torch.tensor(X_exog_train, dtype=torch.float32) if "X_exog_train" in locals() else None
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

# Print tensor shapes
print(f"X_train shape: {X_train_tensor.shape}")
if X_exog_train_tensor is not None:
    print(f"X_exog_train shape: {X_exog_train_tensor.shape}")
print(f"y_train shape: {y_train_tensor.shape}")



In [ ]:
from Evaluation.evaluation_helpers import evaluate_model

# Retrieve best model path from training metadata
best_model_path = trainingMetadata.get("best_model_path")

# Ensure the model file exists before proceeding
if best_model_path and os.path.exists(best_model_path):
    print(f"Loading best model from: {best_model_path}")
    predictions = evaluate_model(model, best_model_path, X_test, None, device, y_scaler)
else:
    raise FileNotFoundError(f"Best model file not found: {best_model_path}")

# Convert y_test Back to Original Scale
y_test_original = y_scaler.inverse_transform(y_test)

# Plot Predictions vs Actual
plt.figure(figsize=(16, 8))
plt.plot(dates[-len(y_test_original):], y_test_original, label="Actual PCEPI", color="blue", linewidth=2)
plt.plot(dates[-len(predictions):], predictions, label="Predicted PCEPI", linestyle="dashed", color="orange", linewidth=2)
plt.xlabel("Date")
plt.ylabel("PCEPI")
plt.title("Final Optimized N-BEATS PCE Prediction (Residual Learning + Fourier Features)")
plt.legend()
plt.grid()
plt.show()

## Evaluation

In [ ]:
# Compute Evaluation Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test_original, predictions)
rmse = np.sqrt(mean_squared_error(y_test_original, predictions))

print(f"Mean Absolute Error (MAE): {mae:.5f}")
print(f"Root Mean Square Error (RMSE): {rmse:.5f}")

In [ ]:
# Compute residual errors
residuals = y_test_original - predictions

# Visualize Residual Errors Over Time
plt.figure(figsize=(16, 8))
plt.plot(dates[-len(residuals):], residuals, label="Residual Errors", color="red", linewidth=2)
plt.axhline(y=0, color='black', linestyle='dashed')  # Reference line at 0
plt.xlabel("Date")
plt.ylabel("Residual Error")
plt.title("Residual Errors Over Time")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plot Histogram of Residuals
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=30, edgecolor='black', color='red', alpha=0.7)
plt.xlabel("Residual Error")
plt.ylabel("Frequency")
plt.title("Histogram of Residuals")
plt.axvline(x=0, color='black', linestyle='dashed')  # Reference line at 0
plt.grid()
plt.show()